# Functions to help schedule events

how to space out purple event?




In [1]:
var importer = require('../Core');
var google = require('googleapis');
var calendar = google.calendar('v3');
var {
    ISODateString, correctCalendarId,
    getOauthClient, getDaysEvents, d3Heatmap
} = importer.import([
    'convert date iso',
    'lookup calendar name',
    'import google calendar api',
    'd3 calendar',
    'get days events'
])

function googlePromise(func) {
    return new Promise((resolve, reject) => func((err, response) => {
        if (err) reject(err);
        try {
        } catch (e) {
            reject(e);
        }
        setTimeout(() => resolve(response), 100);
    })).catch(e => console.log(e));
};

var DAYS = 7; // plus or minus
var options = {
}
          
var now = new Date();
// TODO: batch promises?
function batchPromises() {
    const promises = Array.from(Array(Math.ceil(DAYS / 5)).keys()).map((acc, i) => {
        const daysBeforeAfter = i - Math.ceil(DAYS / 5 / 2);
        return resolve => Promise.all(Array.from(Array(5).keys()).map(day => {
            const date = new Date(now);
            date.setDate(date.getDate() + daysBeforeAfter * 5 + day);
            return getDaysEvents(date, options);
        }))
            .then(r => {
                setTimeout(() => resolve(r), 1000);
            })
    });
    return importer.runAllPromises(promises)
        .then(r => [].concat(...r))
}

function adjustPurpleEvents() {
    return getOauthClient(options)
        .then(() => correctCalendarId(options))
        .then(() => typeof global.listEvents === 'undefined'
              ? batchPromises()
              : Promise.resolve(global.listEvents))
        .then(r => {
            global.listEvents = r;
            const purpleEvents = r.filter(e => e.event.colorId === '1');
            return importer.runAllPromises(purpleEvents.map((e, i) => {
                // assuming I can only get 1 task done per day
                const newTime = new Date(now);
                newTime.setDate(newTime.getDate() + i);
                newTime.setHours(12, 0, 0);
                return googlePromise(
                    calendar.events.patch.bind(calendar.events, {
                        eventId: e.event.id,
                        calendarId: options.calendarId,
                        auth: options.auth,
                        resource: {
                            start: { dateTime: new Date(newTime.getTime()) },
                            end: { dateTime: new Date(newTime.getTime() + 120 * 60 * 1000) },
                        }
                    })).then(r => new Promise(resolve => {
                        setTimeout(() => resolve(r), 1000);
                    }));
                // TODO: skip dates using chrono description relative to each date
                // TODO: skip overlap using study sauce algorithm
                // TODO: skip boundaries using a new algorithm for defining inside and outside times
            }))
        })
        .then(r => r.map((e, i) => ({
            id: e.id,
            start: new Date(e.start.dateTime),
            end: new Date(e.end.dateTime),
        })))
        .then(r => d3Heatmap(r))
}
module.exports = adjustPurpleEvents;

if(typeof $$ !== 'undefined') {
    $$.async();
    adjustPurpleEvents()
        .then(r => $$.html(r))
        .catch(e => $$.sendError(e))
}



reading notebook /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[3]
compiling /Users/briancullinan/jupytangular2/Core/interpret questions.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Utilities/file system.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[5]
reading notebook /Users/briancullinan/jupytangular2/Analytics/budgeting.ipynb
reading notebook /Users/briancullin

2018 Mon Tue Wed Thu Fri Sat Sun Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec 2017 Mon Tue Wed Thu Fri Sat Sun Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec 2016 Mon Tue Wed Thu Fri Sat Sun Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec 2015 <rect fill="rgb(255,255,255)" class="day" width="15.958333333333334" h

how to schedule study time?

how to schedule hours per event?

generalize from study sauce v2


how to schedule inside outside of time bounds?



how to remove overlaps?

how to remove overlaps from events with alloable wiggle room?



how to prioritize events based on glob summary?




calendar search heatmap?



In [ ]:
var importer = require('../Core');
var {
    getOauthClient,
    d3Heatmap,
    listEvents,
    correctCalendarId
} = importer.import([
    'import google calendar api',
    'd3 calendar',
    'list events',
    'lookup calendar name',
]);

var options = {
    
}

function calendarHeatmap(search, calendar) {
    if(calendar) {
        options.calendarId = calendar;
    }
    return getOauthClient(options)
        .then(() => correctCalendarId(options))
        .then(() => listEvents({
            auth: options.auth,
            calendarId: options.calendarId,
            q: search
        }))
        .then(r => r.map((e, i) => ({
            id: e.id,
            start: new Date(e.event.start.dateTime),
            end: new Date(e.event.end.dateTime),
        })))
        .then(r => [d3Heatmap(r)])
}
module.exports = calendarHeatmap;

